In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import ctypes as ct
import scipy.interpolate
from aerofiles.igc import Reader
from IPython.display import clear_output, display
import jxfoil
import pandas as pd
from datetime import datetime

In [3]:
# Function to read airfoil data at each station

def GetAirfoilData(Filename):
    with open(Filename, 'r') as infile:
        AirfoilData = np.loadtxt(infile, dtype = str, skiprows = 0, unpack = True)
    
    return AirfoilData

In [4]:
# Function to read wing geometry

def GetWingData(Filename):
    with open(Filename, 'r') as infile:
        WingData = np.loadtxt(infile, dtype = float, skiprows = 0, unpack = True)
    area = 0
    span = 0
    
    for i in range(0,len(WingData[0]),1):
        span = span + WingData[2][i]
        area = area + ((WingData[0][i] + WingData[1][i])/2)*WingData[2][i]
    
    AR = 2*(span**2)/area
    
    return area, span, AR, WingData

In [5]:
# Function to divide wing into n sections, returns chord lengths array
#NOT USED

def WingDivide(WingData, n):
    number_of_panels = WingData[0].size

    #determine wing span from wing file
    span = 0
    if number_of_panels > 1:
        for panel_length in WingData[5]:
            span = span + panel_length 
    else:
        span = WingData[5]
        
    #print("Wing span => " + str(2*span))
    
    
    # divide wing into n sections
    chords = np.zeros(n, dtype = float)
    current_panel = 0 
    current_span_pos = WingData[5][0]
    i = 0
    for y_pos in np.linspace(0, span, n):
           
        if y_pos > current_span_pos:
            current_panel = current_panel + 1
            current_span_pos = current_span_pos + WingData[5][current_panel]
        
        chords[i] = ((WingData[0][current_panel] - WingData[1][current_panel]) / WingData[5][current_panel]) * (current_span_pos - y_pos) + WingData[1][current_panel]
        i += 1
    #print(np.stack((np.linspace(0,span,n),chords),axis=1))    
    return(np.stack((np.linspace(0, span, n), chords), axis = 1))
 

In [6]:
def Get_Chord(WingData, y_pos):
    chord = 0
    number_of_panels = WingData[0].size

    #determine wing span from wing file
    span = 0
    if number_of_panels > 1:
        for panel_length in WingData[2]:
            span = span + panel_length 
    else:
        span = WingData[2]
        
    #print("Wing half span => " + str(span))
    
    
    current_panel = 0
    current_span_pos = WingData[2][0]   #outer section of current panel
     
    while current_panel <= number_of_panels + 1:
        if y_pos < current_span_pos:
                chord = ((WingData[0][current_panel] - WingData[1][current_panel])/WingData[2][current_panel])*(current_span_pos - y_pos) + WingData[1][current_panel]
                break
        
        current_panel += 1 
        current_span_pos = current_span_pos + WingData[2][current_panel]
               
        #print(current_panel,current_span_pos)
        
    #print(np.stack((np.linspace(0, span, n), chords), axis = 1))    
    return(chord)

In [7]:
def Solve_LL(wingdata, span, AR, a, n):
    
    #a is the lift slope cl/radians  -> 2*PI
    alpha = 5
    theta_range = np.linspace(90, 0.01, n)
    AMatrix = np.ones(shape=(n, n))
    X = np.ones(n)
    delta = 0
    Cl = np.zeros(n)
    y_pos = np.zeros(n)
    
    
    i = 0
    for theta in theta_range:
        unit_y = -math.cos(np.deg2rad(theta)) 
        c = Get_Chord(wingdata, -unit_y*span)
        mu = c*a/(8*span)
        
        X[i] = mu*(np.deg2rad(alpha))*(math.sin(np.deg2rad(theta)))
        for j in range(0,n,1):
            AMatrix[i][j] = (math.sin(((j + 1)*2 - 1)*np.deg2rad(theta)))*(((j + 1)*2 - 1)*mu + (math.sin(np.deg2rad(theta))))
        
        y_pos[i] = -unit_y*span
        i+=1
    
    #print(A)
    #print(X)      
    #A = np.matmul(np.linalg.inv(AMatrix),X)
    A = np.linalg.solve(AMatrix, X)
    
    
    for i in range(1, n, 1):
        delta = delta +((i + 1)*2 - 1)*(A[i]**2)/(A[0]**2)

    #e =  (1/(1+delta))     

    k = 0
    for theta in theta_range:  
        for j in range(0, n, 1):
            unit_y = -math.cos(np.deg2rad(theta))
            c = Get_Chord(wingdata, -unit_y*span)
            Cl[k] = Cl[k] + (8*span)*(A[j]*(math.sin(((j + 1)*2 - 1)*np.deg2rad(theta)))) / c
            
        k+=1
           
    CL = A[0]*(np.pi)*AR
    Cl_unit = Cl/CL
    return(Cl_unit, y_pos, CL, delta) 

In [8]:
def Standard_Atmosphere(FileName, Elevation, standard_rho):
    
    with open(FileName, 'r') as infile:
        S_atm_Data = np.loadtxt(infile, dtype = float, skiprows = 1, unpack = True)
    
    
    pressure_interp = scipy.interpolate.interp1d(S_atm_Data[0], S_atm_Data[2])
    density_interp = scipy.interpolate.interp1d(S_atm_Data[0], S_atm_Data[3])
    Kin_viscosity_interp = scipy.interpolate.interp1d(S_atm_Data[0], S_atm_Data[4])
   
    pressure = pressure_interp(Elevation)
    rho = standard_rho*density_interp(Elevation)
    mu = rho*Kin_viscosity_interp(Elevation)*1e-5
    
    #print(pressure_interp)
    #print(S_atm_Data)
    
    
    return(pressure, rho, mu)

In [9]:
def Get_rho_mu(P_altitude, OAT, rho):
    # standard rho 
    
    P, rho, mu= Standard_Atmosphere("Standard_atmosphere.txt", P_altitude, rho)
    
    R = 287.058
    T = OAT + 273.15
    
    P = P*100000
    
    
    return((P/(R*T)), mu)

In [10]:
def Wing_CD(WingFileName, AirfoilFileName, V, rho, mu, bank_angle, weight, Flap, a):
   
    # get wing definition data
    Area, Span, AR, WingData = GetWingData(WingFileName)
    
    
    
    # calculte span efficiency and cl-distribution
    n = 60
    cl, y_pos, CL_theory, delta = Solve_LL(WingData, Span, AR, a, n)

    #set cl interpolation function 
    cl_interp = scipy.interpolate.interp1d(y_pos, cl)


    CL = (2*weight) / (rho*(V**2)*2*Area*np.cos(np.deg2rad(bank_angle)))

    # calculate profile drag of the wing

    AirfoilData  = GetAirfoilData(AirfoilFileName)

    drag = 0
    area = 0
    span = 0.02

    Re_root = rho*V*float(AirfoilData[1][0])/mu
    AOA, Cd_root, Cm, v = jxfoil.CallXfoilcl(cl_interp(span)*CL, Re_root, AirfoilData[0][0], Flap, 0, 0)

    #print(span,cl_interp(span)*CL,AOA,Cd_root,Re_root)
    for i in range(0, len(WingData[0]), 1):


        Re_tip = rho*V*float(AirfoilData[1][i + 1])/mu

        span = span + 0.995*WingData[2][i]
        area = ((WingData[0][i] + WingData[1][i])/2)*WingData[2][i]

        # call XFOIL DLL


        AOA, Cd_tip, Cm, v = jxfoil.CallXfoilcl(cl_interp(span)*CL, Re_tip, AirfoilData[0][i+1], Flap, 0, 0)
        #print(span, cl_interp(span)*CL, AOA, Cd_tip, Re_tip)


        drag = drag + 0.5*rho*(V**2)*((Cd_root+Cd_tip)/2)*area

        Cd_root = Cd_tip

    CDi = ((CL**2)/(np.pi * AR))*(1 + delta)
    CD = 2*drag/(rho*(V**2)*Area) + CDi
    

    return(CD, CL)

In [11]:
def Fuselage_Drag(FileName, V):
    with open(FileName, 'r') as infile:
        Fuselage_Data = np.loadtxt(infile, dtype = float, skiprows = 1, unpack = True)    
        
    #print(Fuselage_Data)
    
    drag_interp = scipy.interpolate.interp1d(Fuselage_Data[0], Fuselage_Data[1])
    
    #Drag = drag_interp(V)
    
    #Drag is in Newtons
    return(drag_interp(V))

In [12]:
def Flap_Setting(FileName, V, rho, mu, CL):
    Flap_num = 0
    Flap = 0
    Base_chord = 0.62  #[m]
    
    Re = rho*V*Base_chord/mu
    
    #print(Re)
    with open(FileName, 'r') as infile:
        Flap_Data = np.loadtxt(infile, dtype=float, skiprows=0, unpack=True)

        for i in range(1,len(Flap_Data[0]),1):
            Cl = ((Flap_Data[1][i] - Flap_Data[0][i])/(Flap_Data[1][0] - Flap_Data[0][0]))*(Re - Flap_Data[0][0]) +  Flap_Data[0][i]
            #print(i, Cl)
            if CL < Cl:
                Flap_num = i - 1
                break

    
    match Flap_num:
        case 1:
            Flap = -3
        case 2:
            Flap = 0
        case 3: 
            Flap = 5
        case 4:
            Flap = 13.5
        case 5:
            Flap = 16.7
    
    
    
    #Flap = 0
    return(Flap)


In [26]:
# IGC FILE PARCER MODULE

Weight = 550*9.81  #[N]

rho_SL = 1.225
mu = 1.789e-5
alpha = 5   # angle of attack (only for Lifting line solver) [deg]
a = 2*np.pi  # lift curve slope

Flap = 13.5

WingFile = 'JS3_wing_18m.dat'
FuselageFile = 'JS3_Fuselage.dat'
AirfoilsFile = 'JS3_Airfoil_Data1.dat'
Flap_File = 'JS3_Flap_Data.dat'

IGC_File = '153V6JU1.igc'
Output_File = 'OUTPUT.txt'

# Get wing parameters
Area, Span, AR, WingData = GetWingData(WingFile)
print("Area=" + str(Area*2) + "  " + "Span=" + str(Span*2) + "  " + "AR=" + str(AR))

with open(IGC_File, 'r') as f:
	IGC_Data = Reader().read(f)

i = 200
    
TAS_previous = IGC_Data["fix_records"][1][i - 1]["TAS"]                           #True airspeed
AOR_previous = IGC_Data["fix_records"][1][i - 1]["AOR"]                           #Angle of roll
pressure_alt_previous = IGC_Data["fix_records"][1][i - 1]["pressure_alt"]         #Pressure altitude
time_previous = IGC_Data["fix_records"][1][i - 1]["time"]  

    
file = open(Output_File,"w")
y = 1
temp = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Performance = [0, 0, 0, 0, 0, 0, 0]
#while i < (len(IGC_Data['fix_records'][1])-0.95*len(IGC_Data['fix_records'][1])):
while i < len(IGC_Data['fix_records'][1]):
#while i < 1505:

    TAS = IGC_Data["fix_records"][1][i]["TAS"]                           
    AOR = IGC_Data["fix_records"][1][i]["AOR"]                           
    OAT = IGC_Data["fix_records"][1][i]["OAT"]                           
    #VAR = IGC_Data["fix_records"][1][i]["VAR"]                               # Uncompensated variometer 
    pressure_alt = IGC_Data["fix_records"][1][i]["pressure_alt"]         
    time_now = IGC_Data["fix_records"][1][i]["time"]  
    
    V = (TAS_previous + TAS)/720
    Bank_Angle = (AOR_previous + AOR)/2
    #print(i, V)

    FMT = '%H:%M:%S'
    tdelta = datetime.strptime(str(time_now), FMT) - datetime.strptime(str(time_previous), FMT)
    

    timestep = tdelta.seconds
    
    Delta_H = pressure_alt - pressure_alt_previous
  
    
    Sink_V = Delta_H/timestep
 

        
    if (V > 15):   # above 25m/s  do calcs
        rho, mu = Get_rho_mu(pressure_alt, OAT/10, rho_SL)
        
        CL = (2*Weight) /  (rho*(V**2) *2*Area*np.cos(np.deg2rad(Bank_Angle)))
        Flap = Flap_Setting(Flap_File, V, rho, mu, CL)
        
        CD, CL = Wing_CD(WingFile, AirfoilsFile, V, rho, mu, Bank_Angle, Weight, Flap, a)
        #print("CD=" + str(CD) + "  " + "CL=" + str(CL) + " Time: "+ str(time_now))
        
        
        Drag = Fuselage_Drag(FuselageFile, V) + (0.5*rho*(V**2)*CD*Area*2) * 1.1
        Performance_Sink_V = -V/(Weight/Drag)
        
        clear_output(wait=True)
        print("Time: "+ str(time_now))
        print(V*3.6, Sink_V, Performance_Sink_V, Bank_Angle, CD, CL, Flap)
        print(      str('{:.1f}'.format(i/len(IGC_Data['fix_records'][1])*100)) + "% complete"     )
        file.write( "Time: "+ str(time_now) + ",\t Velocity: "+ str('{:.1f}'.format(V*3.6)) + ",\t Sink: "+ str(Sink_V) + ",\t Performance: " + str('{:.3f}'.format(Performance_Sink_V)) + ",\t CL: " +str('{:.3f}'.format(CL)) + ",\t CD: " +str('{:.3f}'.format(CD)) +",\tBank_Angle: " + str(Bank_Angle) + " \t Flap:" + str(Flap)+"\n")
        #file.write( str(datetime.strptime(str(time_now), FMT)) + ","+ str(V) + ","+ str(Sink_V) + "," + str(Performance_Sink_V) + "," + str(VAR/100)+"," +str(Bank_Angle) +"\n")
    else:
        Drag = 0

        Performance_Sink_V

    i += 500

    TAS_previous = TAS
    AOR_previous = AOR
    pressure_alt_previous = pressure_alt
    time_previous = time_now
    
file.close()
print("Done")






Time: 13:48:36
204.97 -1.452 -7.010871005507122 -5.0 0.0290697932258177 0.29087522347051875 -3
96.1% complete
Done


KeyboardInterrupt: 